## Data cleaning the listings data.


In [20]:
import pandas as pd 
import numpy as np 

# load boroughs and their postcodes
listings_df = pd.read_csv("../data/lisings_new.csv")  # Replace with your actual filename


print(listings_df.head())



          id date_listed  bedrooms  bathrooms  \
0  163439072  2025-06-17         0        NaN   
1  163629194  2025-06-21         1        1.0   
2  163629197  2025-06-21         3        2.0   
3  163629167  2025-06-21         2        1.0   
4  162040442  2025-05-16         2        1.0   

                                             summary  \
0  base rent: £1100, deposit: £1269.23, number of...   
1  LONG LET. Arranged on the 3rd floor, this eleg...   
2  LONG LET. Set within a modern residential deve...   
3  A TWO bedroom FIRST FLOOR FLAT located within ...   
4  Located in the quiet a tree lined road in the ...   

                                     address property_type  price_amount  \
0                     Flat 4 27 Seaford Road        Studio          1100   
1    Belsize Park, Belsize Park, London, NW3          Flat          2578   
2  Wilkinson Close, Cricklewood, London, NW2          Flat          2350   
3      Crawley Road, Enfield, Middlesex, EN1          Flat      

## Chunk 1: schema & Data types 

In [48]:
# inspec the current schema and types 
print("\n>> listing_df.info()")


#Convert date_listed to datetime 
listings_df['date_listed'] = pd.to_datetime(listings_df['date_listed'],errors="coerce")
#listings_df.info()



## insure numeric columns are the right dtype
for col in ["bedrooms","bathrooms","price_amount","price_amount","latitude","longitude","page"]:
    listings_df[col] = pd.to_numeric(listings_df[col],errors="coerce")

listings_df['price_amount']= listings_df['price_amount'].astype(float)
listings_df['bathrooms']= listings_df['bathrooms'].astype(int)

for col in ["address","description","borough","property_url","postcode","price_frequency","summary","property_type"]:
    listings_df[col]= listings_df[col].astype(str)


listings_df.info()

print(f" Types {listings_df.dtypes}")

print(listings_df.isna().sum())




>> listing_df.info()
<class 'pandas.core.frame.DataFrame'>
Index: 3307 entries, 0 to 3421
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               3307 non-null   int64         
 1   date_listed      3307 non-null   datetime64[ns]
 2   bedrooms         3307 non-null   int64         
 3   bathrooms        3307 non-null   int64         
 4   summary          3307 non-null   object        
 5   address          3307 non-null   object        
 6   property_type    3307 non-null   object        
 7   price_amount     3307 non-null   float64       
 8   price_frequency  3307 non-null   object        
 9   latitude         3307 non-null   float64       
 10  longitude        3307 non-null   float64       
 11  description      3307 non-null   object        
 12  property_url     3307 non-null   object        
 13  borough          3307 non-null   object        
 14  postcode         3307 n

In [49]:
print(listings_df.dtypes)



id                          int64
date_listed        datetime64[ns]
bedrooms                    int64
bathrooms                   int64
summary                    object
address                    object
property_type              object
price_amount              float64
price_frequency            object
latitude                  float64
longitude                 float64
description                object
property_url               object
borough                    object
postcode                   object
page                        int64
dtype: object


### 2. Handle string naming conventions and trailing spaces


In [54]:
for col in ["address","description","borough","property_url","postcode","price_frequency","summary","property_type"]:
    ## Remove leading / trailing white spaces and replace multiple spaces with single space
    listings_df[col]= listings_df[col].str.strip().str.replace(r'\s+',' ',regex=True)


    ## standardixe casing. 
    #will use title case (Camden Town)
    listings_df[col]= listings_df[col].str.title()

listings_df['postcode']= listings_df['postcode'].str.upper().str.strip()
listings_df['property_url'] = listings_df['property_url'].str.lower().str.strip().str.rstrip('/')




In [ ]:
## find missing values in the whole tables 
print("\n>> Missing values after cleaning:")

#listings_df['bathrooms']= listings_df['bathrooms'].fillna(0)
print(listings_df.isna().sum())

## listing_date is critical for our analysis 
## so drop any nan values 
listings_df= listings_df.dropna(subset='date_listed')
missing_bathrooms = listings_df[listings_df["bathrooms"].isna()]




#print("Missing bathrooms by bedroom count:")
#print(missing_bathrooms["bedrooms"].value_counts())
## We need to check if the missing bathrooms corresponds to studio apartment 
print(f"\n Missing Bathroms property type {missing_bathrooms["property_type"].value_counts()}")




>> Missing values after cleaning:
id                  0
date_listed        68
bedrooms            0
bathrooms          83
summary             0
address             0
property_type       0
price_amount        0
price_frequency     0
latitude            0
longitude           0
description         0
property_url        0
borough             0
postcode            0
page                0
dtype: int64

 Missing Bathroms property type property_type
Flat             31
Studio           15
Apartment        15
Flat Share        2
Ground Flat       2
Not Specified     1
Duplex            1
Terraced          1
Detached          1
Name: count, dtype: int64


In [29]:
print(listings_df.isna().sum())
print(f"Count: {listings_df['id'].count()}")

id                  0
date_listed         0
bedrooms            0
bathrooms          69
summary             0
address             0
property_type       0
price_amount        0
price_frequency     0
latitude            0
longitude           0
description         0
property_url        0
borough             0
postcode            0
page                0
dtype: int64
Count: 3361


 ## 2.Missing Data Analysis
* MCAR (Missing Completely At Random)
* MAR (Missing At Random)
* MNAR (Missing Not At Random)

In [34]:
## create a boolean column markign missing bathrooms 

listings_df["bathrooms_missing"]= listings_df['bathrooms'].isna()

#Check if missingness correlates with bedrooms
print(listings_df.groupby("bathrooms_missing")["bedrooms"].describe())
## no correlations 

# Check if missingness correlates with property_type


# Get overall property_type distribution (percentages)
overall_dist = listings_df['property_type'].value_counts(normalize=True)

# Get property_type distribution within missing bathrooms
missing_dist = listings_df[listings_df['bathrooms'].isna()]['property_type'].value_counts(normalize=True)

# Combine into a dataframe for comparison
missingness_comparison = pd.DataFrame({'Overall': overall_dist, 'Missing Bathrooms': missing_dist}).fillna(0)

print(missingness_comparison)


                    count      mean       std  min  25%  50%  75%  max
bathrooms_missing                                                     
False              3292.0  1.882139  1.224468  0.0  1.0  2.0  2.0  8.0
True                 69.0  1.304348  1.252108  0.0  0.0  1.0  2.0  8.0
                               Overall  Missing Bathrooms
property_type                                            
Apartment                     0.389170           0.217391
Block of Apartments           0.000298           0.000000
Character Property            0.000298           0.000000
Cottage                       0.000298           0.000000
Detached                      0.010414           0.014493
Duplex                        0.003273           0.014493
End of Terrace                0.007438           0.000000
Flat                          0.391253           0.449275
Flat Share                    0.006546           0.028986
Ground Flat                   0.004760           0.028986
House               

In [39]:
## based on the analysis the Studios make up about 5% of all listings but 21.7% of the missing bathrooms. 
# This strong overrepresentation suggests that most missing bathroom values are from studios.
## So, we can fill nans for studios with 0 and drop the rest 
listings_df.loc[(listings_df['bathrooms'].isna()) & (listings_df['property_type']=='Studio'),'bathrooms' ]=0
listings_df=listings_df.dropna(subset=['bathrooms'])
listings_df= listings_df.drop(columns=['bathrooms_missing'])

print(listings_df.isna().sum())

id                 0
date_listed        0
bedrooms           0
bathrooms          0
summary            0
address            0
property_type      0
price_amount       0
price_frequency    0
latitude           0
longitude          0
description        0
property_url       0
borough            0
postcode           0
page               0
dtype: int64


In [56]:
print(listings_df.head(2))

          id date_listed  bedrooms  bathrooms  \
0  163439072  2025-06-17         0          0   
1  163629194  2025-06-21         1          1   

                                             summary  \
0  Base Rent: £1100, Deposit: £1269.23, Number Of...   
1  Long Let. Arranged On The 3Rd Floor, This Eleg...   

                                   address property_type  price_amount  \
0                   Flat 4 27 Seaford Road        Studio        1100.0   
1  Belsize Park, Belsize Park, London, Nw3          Flat        2578.0   

  price_frequency   latitude  longitude     description  \
0         Monthly  51.508900  -0.321970     Studio Flat   
1         Monthly  51.546909  -0.160016  1 Bedroom Flat   

                              property_url borough postcode  page  
0  /properties/163439072#/?channel=res_let  Camden  NW1 2HP     9  
1  /properties/163629194#/?channel=res_let  Camden  NW1 2HP     9  


In [57]:
listings_df.to_csv('list_new_clean.csv',index=False)